## Uncertainty 

1. uncertain about the current state
2. uncertain what the action will do

### Porcine Probability 

In [29]:
from collections import namedtuple
import random

# --------------------------Pig out----------------------------
# high level concept
# play-pig: fun(A, B) -> A
#   - keep scores
#   - take turns
#   - call strategy -> action
#   - do action -> {state}
#   - roll die, + action -> action
# strategy: fun(state) -> action
#
# middle level concept: 
# (1) current state: scores, pending, players,
# (2) actions we can take take: Roll/Hold
#         action: using string 'roll'/'state'
#         roll(state) -> {states}/ hold(state) -> state
#         roll(state, D) -> state //D -> one of {1,2,3,4,5,6} 
#
# low level concept
# (1) the roll of a die: integer
# (2) the implementation of scores: integer
# (3) the implementation of the players and the players to move
#       move: 0/1   players: strategy     
# (4) the goal: integer
#
# States are represented as a tuple of (p, me, you, pending) where
# p:       an int, 0 or 1, indicating which player's turn it is.
# me:      an int, the player-to-move's current score
# you:     an int, the other player's current score.
# pending: an int, the number of points accumulated on current 
# turn, not yet scored
# ---------------------------------------------------------------

# -----------------------------tools-----------------------------
# nametuple:
#     from collections import namedtuple
#     State = namedtuple('state', 'p me you pending')
#     s = State(1,2,3,4)
#     s.p => 1, s.me => 2, s.you => 3, s.pending => 4
#
# ---------------------------------------------------------------

# State = namedtuple('State', 'p me you pending')
other = {1:0, 0:1}                          # mapping from player to other player
possible_moves = ['roll', 'hold']
goal = 50

def hold(state):
    """Apply the hold action to a state to yield a new state:
    Reap the 'pending' points and it becomes the other player's turn."""
    (p, me, you, pending) = state
    return (other[p], you, me + pending, 0)
#     return State(other[state.p], state.you, state.me + state.pending, 0)

def roll(state, d):
    """Apply the roll action to a state (and a die roll d) to yield a new state:
    If d is 1, get 1 point (losing any accumulated 'pending' points),
    and it is the other player's turn. If d > 1, add d to 'pending' points."""
    (p, me, you, pending) = state
    if  d == 1:
        return (other[p], you, me + 1, 0)   # pig out; other player's turn
#         return State(other[state.p], state.you, state.me + 1, 0)
    else:
        return (p, me, you, pending + d)    # accumulate die roll in pending
#         return State(other[state.p], state.me, state.you, state.pending + d)


def hold_at(x):
    """Return a strategy that holds if and only if 
    pending >= x or player reaches goal."""
    def strategy(state):
        (p, me, you, pending) = state
        return 'hold' if (pending >= x or me + pending >= goal) else 'roll'
    strategy.__name__ = 'hold_at(%d)' % x
    return strategy

def clueless(state):
    "A strategy that ignores the state and chooses at random from possible moves."
    return random.choice(possible_moves)

def dierolls():
    "Generator die rolls"
    while True:
        yield random.randint(1,6)

def play_pig(A, B, dierolls = dierolls()):
    """Play a game of pig between two players, represented by their strategies.
    Each time through the main loop we ask the current player for one decision,
    which must be 'hold' or 'roll', and we update the state accordingly.
    When one player's score exceeds the goal, return that player."""
    strategies = [A, B]
    state = (0, 0, 0, 0)
    while True:
        (p, me, you, pending) = state
        if me >= goal :
            return strategies[p]
        elif you >= goal:
            return strategies[other[p]]
        elif strategies[p](state) == 'hold':
            state = hold(state)
        else:
            state = roll(state, next(dierolls))

def always_roll(state):
    return 'roll'

def always_hold(state):
    return 'hold'
    
def test():    
    assert hold((1, 10, 20, 7))    == (0, 20, 17, 0)
    assert hold((0, 5, 15, 10))    == (1, 15, 15, 0)
    
    assert roll((1, 10, 20, 7), 1) == (0, 20, 11, 0)
    assert roll((0, 5, 15, 10), 5) == (0, 5, 15, 15)
    
    assert hold_at(30)((1, 29, 15, 20)) == 'roll'
    assert hold_at(30)((1, 29, 15, 21)) == 'hold'
    assert hold_at(15)((0, 2, 30, 10))  == 'roll'
    assert hold_at(15)((0, 2, 30, 15))  == 'hold'
    
    A, B = hold_at(50), clueless
    rolls = iter([6, 6, 6, 6, 6, 6, 6, 6, 2])
    assert play_pig(A, B, rolls) == A
        
    return 'tests pass'

print (test())

tests pass


In [19]:
from collections import namedtuple
State = namedtuple('State', 'p me you pending')
s = State(1,2,3,4)
s.p

1